# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

Date      id Warehouse       Shipments  \
0  Aug  2 2022  1:22PM  243289        10      8307849-BO   
1  Aug  2 2022  1:22PM  243289        10     8321658-BO1   
2  Aug  2 2022  1:22PM  243289        10  ACI-8308395-BO   
3  Aug  2 2022  1:10PM  243286        20         8351010   
4  Aug  2 2022  1:10PM  243286        20         8351011   
5  Aug  2 2022  1:10PM  243286        20         8351012   
6  Aug  2 2022  1:10PM  243286        20         8351013   
7  Aug  2 2022  1:10PM  243286        20         8351014   
8  Aug  2 2022  1:10PM  243286        20         8351015   
9  Aug  2 2022  1:10PM  243286        20         8351016   

                   Customer ShipmentStatus  
0  ACI Healthcare USA, Inc.       Released  
1  ACI Healthcare USA, Inc.       Released  
2  ACI Healthcare USA, Inc.       Released  
3            Exact-Rx, Inc.       Released  
4            Exact-Rx, Inc.       Released  
5            Exact-Rx, Inc.       Released  
6            Exact-Rx, Inc.       Released  
7            Exact-Rx, Inc.       Released  
8            Exact-Rx, Inc.       Released  
9            Exact-Rx, Inc.       Released

In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

id ShipmentStatus  Shipments
49  243283       Released          9
50  243286       Released          8
51  243287       Released         14
52  243288       Released          1
53  243289       Released          3

In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus  Completed  Executing  Released
id                                            
243283                NaN        NaN       9.0
243286                NaN        NaN       8.0
243287                NaN        NaN      14.0
243288                NaN        NaN       1.0
243289                NaN        NaN       3.0

In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
243170               41.0        4.0       0.0
243173                0.0        1.0      15.0
243174                0.0       14.0      23.0
243204                3.0       15.0       1.0
243208                0.0        2.0      17.0

In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus  Completed  Executing  Released
id                                            
243170                 41          4         0
243173                  0          1        15
243174                  0         14        23
243204                  3         15         1
243208                  0          2        17

In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus      id  Completed  Executing  Released
0               243170         41          4         0
1               243173          0          1        15
2               243174          0         14        23
3               243204          3         15         1
4               243208          0          2        17

In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus      id Completed Executing Released
0               243170        41         4         
1               243173                   1       15
2               243174                  14       23
3               243204         3        15        1
4               243208                   2       17

In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

Date      id Warehouse                         Customer
0   Aug  2 2022  1:22PM  243289        10         ACI Healthcare USA, Inc.
3   Aug  2 2022  1:10PM  243286        20                   Exact-Rx, Inc.
11  Aug  2 2022  1:08PM  243288        10                Yusen – 3D Matrix
12  Aug  2 2022 12:57PM  243287        10                ISDIN Corporation
26  Aug  2 2022 12:50PM  243208        20                   Exact-Rx, Inc.
45  Aug  2 2022 12:38PM  243278        16  American International Chemical
46  Aug  2 2022 12:34PM  243283        10                ISDIN Corporation
55  Aug  2 2022 12:33PM  243279        10                          Bio-PRF
60  Aug  2 2022 12:33PM  243282        10                 Methapharm, Inc.
66  Aug  2 2022 12:33PM  243281        10                 Methapharm, Inc.

In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

Date      id Warehouse                         Customer  \
0  Aug  2 2022  1:22PM  243289        10         ACI Healthcare USA, Inc.   
1  Aug  2 2022  1:10PM  243286        20                   Exact-Rx, Inc.   
2  Aug  2 2022  1:08PM  243288        10                Yusen – 3D Matrix   
3  Aug  2 2022 12:57PM  243287        10                ISDIN Corporation   
4  Aug  2 2022 12:50PM  243208        20                   Exact-Rx, Inc.   
5  Aug  2 2022 12:38PM  243278        16  American International Chemical   
6  Aug  2 2022 12:34PM  243283        10                ISDIN Corporation   
7  Aug  2 2022 12:33PM  243279        10                          Bio-PRF   
8  Aug  2 2022 12:33PM  243282        10                 Methapharm, Inc.   
9  Aug  2 2022 12:33PM  243281        10                 Methapharm, Inc.   

  Completed Executing Released  
0                            3  
1                            8  
2                            1  
3                           14  
4                   2       17  
5                   1           
6                            9  
7                            5  
8                            6  
9                            6

In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

Date      id Warehouse                         Customer  \
0  Aug  2 2022  1:22PM  243289        10         ACI Healthcare USA, Inc.   
1  Aug  2 2022  1:10PM  243286        20                   Exact-Rx, Inc.   
2  Aug  2 2022  1:08PM  243288        10                Yusen – 3D Matrix   
3  Aug  2 2022 12:57PM  243287        10                ISDIN Corporation   
4  Aug  2 2022 12:50PM  243208        20                   Exact-Rx, Inc.   
5  Aug  2 2022 12:38PM  243278        16  American International Chemical   
6  Aug  2 2022 12:34PM  243283        10                ISDIN Corporation   
7  Aug  2 2022 12:33PM  243279        10                          Bio-PRF   
8  Aug  2 2022 12:33PM  243282        10                 Methapharm, Inc.   
9  Aug  2 2022 12:33PM  243281        10                 Methapharm, Inc.   

  Released Executing Completed  
0        3                      
1        8                      
2        1                      
3       14                      
4       17         2            
5                  1            
6        9                      
7        5                      
8        6                      
9        6

In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

Date      id Warehouse                  Customer Released  \
0  Aug  2 2022  1:22PM  243289        10  ACI Healthcare USA, Inc.        3   
1  Aug  2 2022  1:10PM  243286        20            Exact-Rx, Inc.        8   
2  Aug  2 2022  1:08PM  243288        10         Yusen – 3D Matrix        1   
3  Aug  2 2022 12:57PM  243287        10         ISDIN Corporation       14   
4  Aug  2 2022 12:50PM  243208        20            Exact-Rx, Inc.       17   

  Executing Completed  
0                      
1                      
2                      
3                      
4         2

In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Date      id Warehouse                  Customer  Released  \
0  Aug  2 2022  1:22PM  243289        10  ACI Healthcare USA, Inc.       3.0   
1  Aug  2 2022  1:10PM  243286        20            Exact-Rx, Inc.       8.0   
2  Aug  2 2022  1:08PM  243288        10         Yusen – 3D Matrix       1.0   
3  Aug  2 2022 12:57PM  243287        10         ISDIN Corporation      14.0   
4  Aug  2 2022 12:50PM  243208        20            Exact-Rx, Inc.      17.0   

   Executing  Completed  
0        NaN        NaN  
1        NaN        NaN  
2        NaN        NaN  
3        NaN        NaN  
4        2.0        NaN

In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

Date      id Warehouse                  Customer  Released  \
0  Aug  2 2022  1:22PM  243289        10  ACI Healthcare USA, Inc.       3.0   
1  Aug  2 2022  1:10PM  243286        20            Exact-Rx, Inc.       8.0   
2  Aug  2 2022  1:08PM  243288        10         Yusen – 3D Matrix       1.0   
3  Aug  2 2022 12:57PM  243287        10         ISDIN Corporation      14.0   
4  Aug  2 2022 12:50PM  243208        20            Exact-Rx, Inc.      17.0   

   Executing  Completed  
0        0.0        0.0  
1        0.0        0.0  
2        0.0        0.0  
3        0.0        0.0  
4        2.0        0.0

In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

id  Released  Executing  Completed
Warehouse                                         
10         3405772     107.0        0.0        0.0
15         1216113      44.0       30.0        0.0
16          243278       0.0        1.0        0.0
18          243232       1.0        0.0        0.0
19         1945909       7.0       18.0        3.0
20         2189108      64.0       25.0       55.0
21          729737       1.0        2.0        0.0

In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

Warehouse       id  Released  Executing  Completed
0        10  3405772     107.0        0.0        0.0
1        15  1216113      44.0       30.0        0.0
2        16   243278       0.0        1.0        0.0
3        18   243232       1.0        0.0        0.0
4        19  1945909       7.0       18.0        3.0
5        20  2189108      64.0       25.0       55.0
6        21   729737       1.0        2.0        0.0

In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

Warehouse  Released  Executing  Completed
0        10     107.0        0.0        0.0
1        15      44.0       30.0        0.0
2        16       0.0        1.0        0.0
3        18       1.0        0.0        0.0
4        19       7.0       18.0        3.0
5        20      64.0       25.0       55.0
6        21       1.0        2.0        0.0

In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

Warehouse    Status  Value
0        10  Released  107.0
1        15  Released   44.0
2        16  Released    0.0
3        18  Released    1.0
4        19  Released    7.0

In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse     10    15   16   18    19    20   21
Status                                           
Completed    0.0   0.0  0.0  0.0   3.0  55.0  0.0
Executing    0.0  30.0  1.0  0.0  18.0  25.0  2.0
Released   107.0  44.0  0.0  1.0   7.0  64.0  1.0

In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse     Status     10    15   16   18    19    20   21
0          Completed    0.0   0.0  0.0  0.0   3.0  55.0  0.0
1          Executing    0.0  30.0  1.0  0.0  18.0  25.0  2.0
2           Released  107.0  44.0  0.0  1.0   7.0  64.0  1.0

In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

Status     10    15   16   18    19    20   21
0  Completed    0.0   0.0  0.0  0.0   3.0  55.0  0.0
1  Executing    0.0  30.0  1.0  0.0  18.0  25.0  2.0
2   Released  107.0  44.0  0.0  1.0   7.0  64.0  1.0

In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()